In [ ]:
! pip install langchain_community tiktoken langchain-google-genai langchainhub chromadb langchain langchain-huggingface

In [ ]:
import os
os.environ['LANGCHAIN_TRACING_V2'] = 'true'
os.environ['LANGCHAIN_ENDPOINT'] = 'https://api.smith.langchain.com'
os.environ['LANGSMITH_PROJECT'] = "langsmith_project_MY_FIRST_RAG_EVER"
os.environ['LANGCHAIN_API_KEY'] = ""

In [ ]:
os.environ["GOOGLE_API_KEY"] = ""

In [3]:
#### INDEXING ####

# Load blog
import bs4
from langchain import hub
from langchain_core.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate
from langchain_community.document_loaders import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma
from langchain.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from langchain_core.output_parsers import StrOutputParser
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.load import dumps, loads
from operator import itemgetter


loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
blog_docs = loader.load()

# Split
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=300,
    chunk_overlap=50)

# Make splits
splits = text_splitter.split_documents(blog_docs)

# Index

vectorstore = Chroma.from_documents(documents=splits,
                                    embedding=HuggingFaceEmbeddings(model_name = "sentence-transformers/all-mpnet-base-v2"))

retriever = vectorstore.as_retriever()

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [4]:
# Multi Query: Different Perspectives
template = """You are an AI language model assistant. Your task is to generate five
different versions of the given user question to retrieve relevant documents from a vector
database. By generating multiple perspectives on the user question, your goal is to help
the user overcome some of the limitations of the distance-based similarity search.
Provide these alternative questions separated by newlines. Original question: {question}"""
prompt_perspectives = ChatPromptTemplate.from_template(template)


generate_queries = (
    prompt_perspectives
    | ChatGoogleGenerativeAI(model = "gemini-1.5-pro", temperature=0)
    | StrOutputParser()
    | (lambda x: x.split("\n"))
)

In [5]:
def get_unique_union(documents: list[list]):
    """ Unique union of retrieved docs """
    # Flatten list of lists, and convert each Document to string
    flattened_docs = [dumps(doc) for sublist in documents for doc in sublist]
    # Get unique documents
    unique_docs = list(set(flattened_docs))
    # Return
    return [loads(doc) for doc in unique_docs]

# Retrieve
question = "What is task decomposition for LLM agents?"
retrieval_chain = generate_queries | retriever.map() | get_unique_union
docs = retrieval_chain.invoke({"question":question})
len(docs)

<ipython-input-5-ace9fd6455cb>:8: LangChainBetaWarning: The function `loads` is in beta. It is actively being worked on, so the API may change.
  return [loads(doc) for doc in unique_docs]


11

In [7]:

# RAG
template = """Answer the following question based on this context:

{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

llm = ChatGoogleGenerativeAI(model="gemini-1.5-pro", temperature=0)

final_rag_chain = (
    {"context": retrieval_chain,
     "question": itemgetter("question")}
    | prompt
    | llm
    | StrOutputParser()
)

final_rag_chain.invoke({"question":question})

'Task decomposition is the process of breaking down a complex task into smaller, more manageable subtasks.  For LLM agents, this is a crucial step in planning and problem-solving.  The article mentions several approaches to task decomposition:\n\n1. **Chain of Thought (CoT) prompting:** Instructing the LLM to "think step by step," which helps it decompose the task into smaller, simpler steps.\n2. **Tree of Thoughts:** An extension of CoT that explores multiple reasoning possibilities at each step, creating a tree-like structure of potential solutions.\n3. **Simple prompting:** Using prompts like "Steps for XYZ.\\n1." or "What are the subgoals for achieving XYZ?"\n4. **Task-specific instructions:**  Providing instructions tailored to the specific task, such as "Write a story outline" for writing a novel.\n5. **Human input:**  A human can manually decompose the task into subtasks for the LLM.'

In [6]:
# RAG-Fusion: Related
template = """You are a helpful assistant that generates multiple search queries based on a single input query. \n
Generate multiple search queries related to: {question} \n
Output (4 queries):"""
prompt_rag_fusion = ChatPromptTemplate.from_template(template)

In [13]:
generate_queries = (
    prompt_rag_fusion
    | ChatGoogleGenerativeAI(model = "gemini-1.5-flash", temperature=0)
    | StrOutputParser()
    | (lambda x: x.split("\n"))
)

In [14]:
def reciprocal_rank_fusion(results: list[list], k=60):
    """ Reciprocal_rank_fusion that takes multiple lists of ranked documents
        and an optional parameter k used in the RRF formula """

    # Initialize a dictionary to hold fused scores for each unique document
    fused_scores = {}

    # Iterate through each list of ranked documents
    for docs in results:
        # Iterate through each document in the list, with its rank (position in the list)
        for rank, doc in enumerate(docs):
            # Convert the document to a string format to use as a key (assumes documents can be serialized to JSON)
            doc_str = dumps(doc)
            # If the document is not yet in the fused_scores dictionary, add it with an initial score of 0
            if doc_str not in fused_scores:
                fused_scores[doc_str] = 0
            # Retrieve the current score of the document, if any
            previous_score = fused_scores[doc_str]
            # Update the score of the document using the RRF formula: 1 / (rank + k)
            fused_scores[doc_str] += 1 / (rank + k)

    # Sort the documents based on their fused scores in descending order to get the final reranked results
    reranked_results = [
        (loads(doc), score)
        for doc, score in sorted(fused_scores.items(), key=lambda x: x[1], reverse=True)
    ]

    # Return the reranked results as a list of tuples, each containing the document and its fused score
    return reranked_results

retrieval_chain_rag_fusion = generate_queries | retriever.map() | reciprocal_rank_fusion
docs = retrieval_chain_rag_fusion.invoke({"question": question})
len(docs)

6

In [15]:
# RAG
template = """Answer the following question based on this context:

{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

final_rag_chain = (
    {"context": retrieval_chain_rag_fusion,
     "question": itemgetter("question")}
    | prompt
    | llm
    | StrOutputParser()
)

final_rag_chain.invoke({"question":question})

'Task decomposition is the process of breaking down a complex task into smaller, more manageable subgoals.  For LLM agents, this is crucial for effective planning and execution.  LLMs can achieve task decomposition through various methods:\n\n1. **Simple prompting:**  Instructions like "Steps for XYZ:\\n1." or "What are the subgoals for achieving XYZ?" can guide the LLM to break down the task.\n2. **Task-specific instructions:**  Prompts tailored to the task, such as "Write a story outline" for novel writing, can elicit a decomposition relevant to the specific domain.\n3. **Human input:**  Users can provide explicit decomposition steps to the LLM.\n\nTwo prominent techniques related to task decomposition are:\n\n* **Chain of Thought (CoT):**  Prompting the model to "think step by step," decomposing the task into smaller, simpler steps, and providing insights into the model\'s reasoning process.\n* **Tree of Thoughts (ToT):**  Extending CoT by exploring multiple reasoning possibilities 

## Part 7: Decomposition

In [16]:
# Decomposition
template = """You are a helpful assistant that generates multiple sub-questions related to an input question. \n
The goal is to break down the input into a set of sub-problems / sub-questions that can be answers in isolation. \n
Generate multiple search queries related to: {question} \n
Output (3 queries):"""
prompt_decomposition = ChatPromptTemplate.from_template(template)

In [19]:
# LLM
llm = ChatGoogleGenerativeAI(model = 'gemini-1.0-pro', temperature=0)

# Chain
generate_queries_decomposition = ( prompt_decomposition | llm | StrOutputParser() | (lambda x: x.split("\n")))

# Run
question = "What are the main components of an LLM-powered autonomous agent system?"
questions = generate_queries_decomposition.invoke({"question":question})

In [20]:
questions

['1. What are the core components of an LLM-powered autonomous agent system?',
 '2. How do LLMs contribute to the decision-making process in autonomous agents?',
 '3. What are the key challenges in integrating LLMs into autonomous agent systems?']

In [21]:
# Prompt
template = """Here is the question you need to answer:

\n --- \n {question} \n --- \n

Here is any available background question + answer pairs:

\n --- \n {q_a_pairs} \n --- \n

Here is additional context relevant to the question:

\n --- \n {context} \n --- \n

Use the above context and any background question + answer pairs to answer the question: \n {question}
"""

decomposition_prompt = ChatPromptTemplate.from_template(template)

In [22]:
def format_qa_pair(question, answer):
    """Format Q and A pair"""

    formatted_string = ""
    formatted_string += f"Question: {question}\nAnswer: {answer}\n\n"
    return formatted_string.strip()

# llm
llm = ChatGoogleGenerativeAI(model = "gemini-1.0-pro", temperature=0)

q_a_pairs = ""
for q in questions:

    rag_chain = (
    {"context": itemgetter("question") | retriever,
     "question": itemgetter("question"),
     "q_a_pairs": itemgetter("q_a_pairs")}
    | decomposition_prompt
    | llm
    | StrOutputParser())

    answer = rag_chain.invoke({"question":q,"q_a_pairs":q_a_pairs})
    q_a_pair = format_qa_pair(q,answer)
    q_a_pairs = q_a_pairs + "\n---\n"+  q_a_pair

In [23]:
answer

'1. **Finite context length:** LLMs have a limited context capacity, which can make it difficult to integrate them into autonomous agent systems that require access to a large amount of historical information or detailed instructions.\n2. **Challenges in long-term planning and task decomposition:** LLMs can struggle to plan over a lengthy history and effectively explore the solution space. This can make them less robust compared to humans who learn from trial and error.\n3. **Reliability of natural language interface:** Current agent systems rely on natural language as an interface between LLMs and external components such as memory and tools. However, the reliability of model outputs is questionable, as LLMs may make formatting errors and occasionally exhibit rebellious behavior (e.g. refuse to follow an instruction).'

In [24]:
# RAG prompt
prompt_rag = hub.pull("rlm/rag-prompt")

def retrieve_and_rag(question,prompt_rag,sub_question_generator_chain):
    """RAG on each sub-question"""

    # Use our decomposition /
    sub_questions = sub_question_generator_chain.invoke({"question":question})

    # Initialize a list to hold RAG chain results
    rag_results = []

    for sub_question in sub_questions:

        # Retrieve documents for each sub-question
        retrieved_docs = retriever.get_relevant_documents(sub_question)

        # Use retrieved documents and sub-question in RAG chain
        answer = (prompt_rag | llm | StrOutputParser()).invoke({"context": retrieved_docs,
                                                                "question": sub_question})
        rag_results.append(answer)

    return rag_results,sub_questions

# Wrap the retrieval and RAG process in a RunnableLambda for integration into a chain
answers, questions = retrieve_and_rag(question, prompt_rag, generate_queries_decomposition)

<ipython-input-24-b102bf06ba4a>:16: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  retrieved_docs = retriever.get_relevant_documents(sub_question)


In [25]:
def format_qa_pairs(questions, answers):
    """Format Q and A pairs"""

    formatted_string = ""
    for i, (question, answer) in enumerate(zip(questions, answers), start=1):
        formatted_string += f"Question {i}: {question}\nAnswer {i}: {answer}\n\n"
    return formatted_string.strip()

context = format_qa_pairs(questions, answers)

# Prompt
template = """Here is a set of Q+A pairs:

{context}

Use these to synthesize an answer to the question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

final_rag_chain = (
    prompt
    | llm
    | StrOutputParser()
)

final_rag_chain.invoke({"context":context,"question":question})

"The main components of an LLM-powered autonomous agent system are planning, memory, and execution. Planning involves breaking down large tasks into smaller, manageable subgoals and reflecting on past actions to refine future steps. Memory stores information about the agent's past experiences and the current state of the world. Execution involves carrying out the agent's plans and interacting with the environment."

In [26]:
# Few Shot Examples
examples = [
    {
        "input": "Could the members of The Police perform lawful arrests?",
        "output": "what can the members of The Police do?",
    },
    {
        "input": "Jan Sindel’s was born in what country?",
        "output": "what is Jan Sindel’s personal history?",
    },
]
# We now transform these to example messages
example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{input}"),
        ("ai", "{output}"),
    ]
)
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """You are an expert at world knowledge. Your task is to step back and paraphrase a question to a more generic step-back question, which is easier to answer. Here are a few examples:""",
        ),
        # Few shot examples
        few_shot_prompt,
        # New question
        ("user", "{question}"),
    ]
)

In [28]:
generate_queries_step_back = prompt | ChatGoogleGenerativeAI(model = "gemini-1.5-pro", temperature=0) | StrOutputParser()
question = "What is task decomposition for LLM agents?"
generate_queries_step_back.invoke({"question": question})

'What are some common techniques used with LLM agents?'

In [32]:
# Response prompt
response_prompt_template = """You are an expert of world knowledge. I am going to ask you a question. Your response should be comprehensive and not contradicted with the following context if they are relevant. Otherwise, ignore them if they are not relevant.

# {normal_context}
# {step_back_context}

# Original Question: {question}
# Answer:"""
response_prompt = ChatPromptTemplate.from_template(response_prompt_template)

chain = (
    {
        # Retrieve context using the normal question
        "normal_context": RunnableLambda(lambda x: x["question"]) | retriever,
        # Retrieve context using the step-back question
        "step_back_context": generate_queries_step_back | retriever,
        # Pass on the question
        "question": lambda x: x["question"],
    }
    | response_prompt
    | ChatGoogleGenerativeAI(model = "gemini-1.5-pro", temperature=0)
    | StrOutputParser()
)

chain.invoke({"question": question})

'Task decomposition for LLM agents is the process of breaking down a complex task into smaller, more manageable subtasks. This is crucial for agents to effectively handle intricate problems that require multiple steps.  It allows the LLM to focus on more specific and achievable goals, ultimately leading to better overall performance and problem-solving capabilities.  There are several approaches to task decomposition:\n\n* **Chain of Thought (CoT):** This prompting technique encourages the LLM to "think step by step," explicitly outlining the intermediate steps required to reach the final solution.  By decomposing the problem into smaller, simpler steps, CoT improves the LLM\'s ability to reason through complex tasks and provides insights into its reasoning process.\n\n* **Tree of Thoughts (ToT):**  An extension of CoT, ToT explores multiple reasoning possibilities at each step. It generates a tree-like structure where each node represents a thought or a potential solution at a given s

In [4]:
# HyDE document genration
template = """Please write a scientific paper passage to answer the question
Question: {question}
Passage:"""
prompt_hyde = ChatPromptTemplate.from_template(template)

generate_docs_for_retrieval = (
    prompt_hyde | ChatGoogleGenerativeAI(model = "gemini-1.0-pro", temperature=0) | StrOutputParser()
)

# Run
question = "What is task decomposition for LLM agents?"
generate_docs_for_retrieval.invoke({"question":question})

"**Task Decomposition for Large Language Models (LLMs)**\n\n**Introduction**\n\nLarge Language Models (LLMs) have demonstrated remarkable capabilities in various natural language processing tasks. However, they often struggle with complex tasks that require reasoning and decomposition. Task decomposition is a technique that breaks down a complex task into smaller, more manageable subtasks. This approach can enhance the performance of LLMs by providing them with a structured framework for solving problems.\n\n**Definition of Task Decomposition**\n\nTask decomposition involves identifying the individual steps or subtasks that are necessary to complete a given task. These subtasks can be either sequential or parallel, and they may require different skills or knowledge. By decomposing a task, we create a hierarchical structure that guides the LLM's reasoning process.\n\n**Benefits of Task Decomposition**\n\nTask decomposition offers several benefits for LLM agents:\n\n* **Improved Reasonin

In [5]:
# Retrieve
llm = ChatGoogleGenerativeAI(model = "gemini-1.0-pro", temperature=0)
retrieval_chain = generate_docs_for_retrieval | retriever
retrieved_docs = retrieval_chain.invoke({"question":question})
retrieved_docs

[Document(metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}, page_content='LLM Powered Autonomous Agents\n    \nDate: June 23, 2023  |  Estimated Reading Time: 31 min  |  Author: Lilian Weng\n\n\nBuilding agents with LLM (large language model) as its core controller is a cool concept. Several proof-of-concepts demos, such as AutoGPT, GPT-Engineer and BabyAGI, serve as inspiring examples. The potentiality of LLM extends beyond generating well-written copies, stories, essays and programs; it can be framed as a powerful general problem solver.\nAgent System Overview#\nIn a LLM-powered autonomous agent system, LLM functions as the agent’s brain, complemented by several key components:\n\nPlanning\n\nSubgoal and decomposition: The agent breaks down large tasks into smaller, manageable subgoals, enabling efficient handling of complex tasks.\nReflection and refinement: The agent can do self-criticism and self-reflection over past actions, learn from mistakes and refi

In [6]:
# RAG
template = """Answer the following question based on this context:

{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

final_rag_chain = (
    prompt
    | llm
    | StrOutputParser()
)

final_rag_chain.invoke({"context":retrieved_docs,"question":question})

'Task decomposition for LLM agents involves breaking down large tasks into smaller, manageable subgoals, enabling efficient handling of complex tasks.'